In [ ]:
#  手写识别案例

In [ ]:
##  利用汉明距离方法求解

In [ ]:
##  1、准备数据

In [1]:
from os import listdir
import pandas as pd
import numpy as np

In [2]:
"""
函数功能：得到标记好的数据集
参数介绍：
--filename:训练集和测试集的存放路径
"""
def get_train_test(filepath):
    path = filepath
    Filelist = listdir(path)
    data = pd.DataFrame()
    img = []
    labels = []
    for i in range(len(Filelist)):
        filename = Filelist[i]
        txt = pd.read_csv(path +'/'+filename,header = None)
        num = ''
        for i in range(txt.shape[0]):
            num += txt.iloc[i,:]
        img.append(num[0])
        filelabels = filename.split('_')[0]
        labels.append(filelabels)
    data['img'] = img
    data['num'] = labels
    return data
    
    

In [52]:
# path = 'trainingDigits'
path = 'F:/jupyter/data'
Filelist = listdir(path)
filename = Filelist[0]
print(path +'/'+filename)
# txt = pd.read_csv(path +'/'+filename,header = None)
import csv
txt = csv.reader(open(path+'/'+filename)) 
print(txt)
# num = ''
# for i in range(txt.shape[0]):
#     num += txt.iloc[i,:]
# print(num)
# img = []
# img.append(num[0])
# filelabels = filename.split('_')[0]
# # data = get_train_test('trainingDigits')
data = get_train_test('F:/jupyter/data/breast-data.csv')
print(data)

F:/jupyter/data/breast-data.csv


NotADirectoryError: [WinError 267] 目录名称无效。: 'F:/jupyter/data/breast-data.csv'

In [35]:
# path = 'F:/jupyter/机器学习/data/breast-data.csv'
# txt = pd.read_csv(path,encoding='utf-8',header=None,sep = '\t')
# print(txt)
# # Filelist = listdir(path)
# # print(Filelist)
# # filename = Filelist[0]
# # print(filename)

OSError: Initializing from file failed

In [ ]:
np.unique(data.num)

In [ ]:
train = get_train_test('trainingDigits')
test = get_train_test('testDigits')

In [ ]:
train.head(1)

In [ ]:
test.head(1)

In [ ]:
from Levenshtein import hamming
def handwritingClass(train,test,k):
    n = train.shape[0]
    m = test.shape[0]
    result = []
    for i in range(m):
        dist = []
        for j in range(n):
            d = str(hamming(train.iloc[j,0],test.iloc[i,0]))
            dist.append(d)
        dist_l = pd.DataFrame({'dist':dist,'labels':(train.iloc[:,1])})
        dr = dist_l.sort_values(by = 'dist')[: k]
        re = dr.loc[:, 'labels'].value_counts()
        result.append(re.index[0])
    result = pd.Series(result)
    test['predict'] = result
    acc = (test.iloc[:,-1]==test.iloc[:,-2]).mean()
    return test

In [ ]:
test = handwritingClass(train,test,4)

In [ ]:
test.num.values

In [ ]:
(test.predict.values == test.num.values).mean()

##  利用欧式距离方法求解

In [43]:
"""
函数说明:将32x32的二进制图像转换为1x1024向量。

Parameters:
    filename - 文件名
Returns:
    returnVect - 返回的二进制图像的1x1024向量

"""
def img2vector(filename):
    #创建1x1024零向量
    returnVect = np.zeros((1, 1024))
    #打开文件
    fr = open(filename)
    #按行读取
    for i in range(32):
        #读一行数据
        lineStr = fr.readline()
        #每一行的前32个元素依次添加到returnVect中
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])
    #返回转换后的1x1024向量
    return returnVect


In [53]:
returnVect = np.zeros((1, 1024))
returnVect
fr = open('F:/jupyter/data/low-breast-cancer.data')
fr.readline()
returnVect = img2vector('F:/jupyter/data/low-breast-cancer.data')
returnVect

ValueError: invalid literal for int() with base 10: 'n'

In [54]:
def train_test_data(filename):
    Labels = []#标签
    #返回trainingDigits,testDigits目录下的文件名
    FileList = listdir(filename)
    #返回文件夹下文件的个数
    m = len(FileList)
    #初始化训练的Mat矩阵,训练测试集
    Mat = np.zeros((m, 1024))
    #从文件名中解析出训练集的类别
    for i in range(m):
        #获得文件的名字
        fileNameStr = FileList[i]
        #获得分类的数字
        classNumber = int(fileNameStr.split('_')[0])
        #将获得的类别添加到hwLabels中
        Labels.append(classNumber)
        #将每一个文件的1x1024数据存储到trainingMat矩阵中
        Mat[i,:] = img2vector('%s/%s' % (filename,fileNameStr))
    return Mat,Labels

In [55]:
trainingMat,xlLabels = train_test_data('trainingDigits')#训练集
vectorUnderTest,csLabels = train_test_data('testDigits')#测试集

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'trainingDigits'

In [ ]:
def classify0(inX, dataSet, labels, k):
    #numpy函数shape[0]返回dataSet的行数
    dataSetSize = dataSet.shape[0]
    #在列向量方向上重复inX共1次(横向),行向量方向上重复inX共dataSetSize次(纵向)
    diffMat = np.tile(inX, (dataSetSize, 1)) - dataSet
    #二维特征相减后平方
    sqDiffMat = diffMat**2
    #sum()所有元素相加,sum(0)列相加,sum(1)行相加
    sqDistances = sqDiffMat.sum(axis=1)
    #开方,计算出距离
    distances = sqDistances**0.5
    #返回distances中元素从小到大排序后的索引值
    sortedDistIndices = distances.argsort()
    #定一个记录类别次数的字典
    classCount = {}
    for i in range(k):
        #取出前k个元素的类别
        voteIlabel = labels[sortedDistIndices[i]]
        #dict.get(key,default=None),字典的get()方法,返回指定键的值,如果值不在字典中返回默认值。
        #计算类别次数
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1
    #python3中用items()替换python2中的iteritems()
    #key=operator.itemgetter(1)根据字典的值进行排序
    #key=operator.itemgetter(0)根据字典的键进行排序
    #reverse降序排序字典
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    #返回次数最多的类别,即所要分类的类别
    return sortedClassCount[0][0]

In [ ]:
"""
函数说明:手写数字分类测试
"""
import operator
def handwritingClassTest():
    #测试集的Labels
    hwLabels = []
    #返回trainingDigits目录下的文件名
    trainingFileList = listdir('trainingDigits')
    #返回文件夹下文件的个数
    m = len(trainingFileList)
    #初始化训练的Mat矩阵,测试集
    trainingMat = np.zeros((m, 1024))
    #从文件名中解析出训练集的类别
    for i in range(m):
        #获得文件的名字
        fileNameStr = trainingFileList[i]
        #获得分类的数字
        classNumber = int(fileNameStr.split('_')[0])
        #将获得的类别添加到hwLabels中
        hwLabels.append(classNumber)
        #将每一个文件的1x1024数据存储到trainingMat矩阵中
        trainingMat[i,:] = img2vector('trainingDigits/%s' % (fileNameStr))
    #返回testDigits目录下的文件名
    testFileList = listdir('testDigits')
    #错误检测计数
    errorCount = 0.0
    #测试数据的数量
    mTest = len(testFileList)
    #从文件中解析出测试集的类别并进行分类测试
    for i in range(mTest):
        #获得文件的名字
        fileNameStr = testFileList[i]
        #获得分类的数字
        classNumber = int(fileNameStr.split('_')[0])
        #获得测试集的1x1024向量,用于训练
        vectorUnderTest = img2vector('testDigits/%s' % (fileNameStr))
        #获得预测结果
        classifierResult = classify0(vectorUnderTest, trainingMat, hwLabels, 3)
        #print("分类返回结果为%d\t真实结果为%d" % (classifierResult, classNumber))
        if(classifierResult != classNumber):
            errorCount += 1.0
    print("总共错了%d个数据\n错误率为%f%%" % (errorCount, errorCount/mTest))

In [ ]:
handwritingClassTest()

In [ ]:
(946-11)/946